Importing Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import os
import cv2
import pickle
import random
import sklearn
import glob

In [ ]:

from tensorflow.keras.layers import Dense,Activation,Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.models import Sequential
from matplotlib.image import imread
import matplotlib.pyplot as plt
import tensorflow as tf

Data Path

In [ ]:
train = r'C:/Users/Lenovo/OneDrive/Desktop/ML/Data_Sets/Tomatod/tomato/train'
test = r'D:/ML/Data_sets/Tomatod/tomato/val'
size=256

Data Augumentation

In [ ]:
datagen=ImageDataGenerator(rescale=1./255,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                vertical_flip=True,
                                validation_split=0.2)



In [ ]:
X_train = ImageDataGenerator(rescale=1./255).flow_from_directory(train,
                                      target_size=(size,size),
                                      batch_size=32,
                                      class_mode='categorical',
                                      subset='training')
X_test = ImageDataGenerator(rescale=1./255).flow_from_directory(test,
                                     target_size=(size,size),
                                     batch_size=32,
                                     class_mode='categorical',
                                     subset='training')


In [ ]:
lables = X_test.class_indices.keys()

In [ ]:
print(lables)

Call back Setup


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

checkpoint=ModelCheckpoint(r'model70epochs.h5',
                          monitor='val_loss',
                          mode='min',
                          save_best_only=True,
                          verbose=1)
earlystop=EarlyStopping(monitor='val_loss',
                       min_delta=0,
                       patience=5,
                       verbose=1,
                       restore_best_weights=True)
callbacks=[checkpoint,earlystop]

Modelling Starts

In [ ]:
feature_extractor=Sequential()

feature_extractor.add(Conv2D(64,(3,3),activation='relu',padding='same',input_shape=(size,size,3)))
feature_extractor.add(MaxPooling2D(2,2))
feature_extractor.add(Conv2D(32,(3,3),activation='relu',padding='same'))
feature_extractor.add(MaxPooling2D(2,2))
feature_extractor.add(Conv2D(16,(3,3),activation='relu',padding='same'))
feature_extractor.add(MaxPooling2D(2,2))
feature_extractor.add(Flatten())



In [ ]:
x = feature_extractor.output
x = Dense(128, activation = 'relu',kernel_initializer = 'he_uniform')(x)
prediction_layer = Dense(5,activation='softmax')(x)

In [ ]:
train_images = []
train_labels = []
for directory_path in glob.glob('D:/ML/Data_sets/Tomatod/tomato/train/*'):
    label = directory_path.split("\\")[-1]
    #print(label)
    for img_path in glob.glob(os.path.join(directory_path,"*.jpg")):
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img,(256,256))
        train_images.append(img)
        train_labels.append(label)

In [ ]:
test_images = []
test_labels = []
for directory_path in glob.glob('D:/ML/Data_sets/Tomatod/tomato/val/*'):
    label = directory_path.split("\\")[-1]
    #print(label)
    for img_path in glob.glob(os.path.join(directory_path,"*.jpg")):
        #print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img,(256,256))
        test_images.append(img)
        test_labels.append(label)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [ ]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [ ]:
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
x_train , y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
x_train, x_test = x_train / 255.0 , x_test / 255.0

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix,accuracy_score


In [ ]:
x_RF = feature_extractor.predict(x_train)

In [ ]:
print(x_RF.shape)

In [ ]:
RF_model = RandomForestClassifier(n_estimators=50, random_state=42)

In [ ]:
RF_model.fit(x_RF,y_train)

In [ ]:
x_test_feature = feature_extractor.predict(x_test)

In [ ]:
prediction_RF = RF_model.predict(x_test_feature)

In [ ]:
prediction_RF = le.inverse_transform(prediction_RF)

In [ ]:
print("Accuracy => ", accuracy_score(test_labels,prediction_RF)*100)

In [ ]:
cm = confusion_matrix(test_labels, prediction_RF)

In [ ]:
pd.crosstab(test_labels, prediction_RF)


Testing the model

In [ ]:

imgT = "D:/ML/Data_sets/Tomatod/tomato/test/sept.JPG"
imgT1 = cv2.imread(imgT)
imgT1 = imgT1/255.0

input_img = np.expand_dims(imgT1, axis=0)
input_img_features = feature_extractor.predict(input_img)
prediction_RF = RF_model.predict(input_img_features)[0]
prediction_RF = le.inverse_transform([prediction_RF])
plt.imshow(imgT1)
plt.title(f"Predicted as : {prediction_RF}")



**Saving the Random Forest Model**

In [ ]:
import pickle

In [ ]:
saved_model = 'CNNandRF.h5'
pickle.dump(RF_model,open(saved_model,'wb'))

In [ ]:
filename = 'C:/Users/Lenovo/OneDrive/Desktop/ML/Project_Src/CNNandRF.h5'
loaded_model = pickle.load(open(filename,'rb'))

Saving Feature extracted model


In [ ]:
feature_extracted_model = 'extracted_features.h5'
pickle.dump(feature_extractor,open(feature_extracted_model,'wb'))

In [ ]:
feature_file = 'C:/Users/Lenovo/OneDrive/Desktop/ML/Project_Src/extracted_features.h5'
loaded_feature = pickle.load(open(feature_file,'rb'))

Testing loaded model

In [ ]:

imgT = "D:/ML/Data_sets/Tomatod/tomato/test/healthy.JPG"
imgT1 = cv2.imread(imgT)
imgT1 = imgT1/255.0

input_img = np.expand_dims(imgT1, axis=0)
input_img_features = loaded_feature.predict(input_img)
prediction_RF = loaded_model.predict(input_img_features)[0]
results = {0:"Bacterial Spot",1:"Late Blight",2:"Septoria Leaf Spot",4:"Healthy"}
prediction_RF = results[prediction_RF]
plt.imshow(imgT1)
plt.title(f"Predicted as : {prediction_RF}")